# LOAD PROQUEST HTML DATA

This notebook enables you to upload one or more HTML results files from searches performed in ProQuest. The files are converted into valid JSON manifests, allowing the rest of the Virtual Workspace to work with them.

Run the **Settings** and **Basic Setup** cells _before_ uploading your ProQuest HTML files.

## SETTINGS

In [ ]:
import re
import os
import glob
import json
import shutil
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil import parser as dateparser
from IPython.display import display, HTML

project_dir = %pwd
proquest_dir = 'caches/proquest_sources'
results_dir = 'caches/json'

## BASIC SETUP

This cell creates the required directory into which you will upload your ProQuest HTML file(s) and the directory into which the json files you create will be put. Please run this cell and then follow the instructions for uploading.

In [ ]:
!rm -r caches/proquest_sources # if this is the first time you have run this cell, this will create output that reads
# rm: cannot remove 'caches/proquest_sources': No such file or directory
# this is normal and expected
!mkdir -p caches/proquest_sources
!rm -r caches/json
!mkdir -p caches/json

msg = '''
<h3>You are now ready to upload your material.</h3>
<p>Navigate to the <code>caches/proquest_sources</code> folder in this project's directory and click the "Upload" button to upload each ProQuest HTML file you want to import.
<p>Note that this data should be in .html format.</p>
'''
output = HTML(msg)

display(output)

## FILENAMES: Determine the filenames of the ProQuest HTML files

In [ ]:
datafile_list = []
for (dirname, _dirs, files) in os.walk(proquest_dir):
    for filename in files:
        if filename.endswith('.html'):
            filepath = os.path.join(dirname.split(proquest_dir)[1], filename)
            datafile_list.append(filepath)

## CREATE JSON: Create the JSON files

This works by chopping up single html file into separate chunks according to a set delimiter. Then, we use a monstrous for loop to strip info from each html file using combo of Beautiful soup and regex. We put this data into a dictionary. This is messy because ProQuest html files are very poorly tagged. We then dump this dictionary into json files in the project's json directory. We save each json file with a padded id as the filename in a subdirectory determined by the original ProQuest file it came from (so, all individual articles from the same original ProQuest results html file will be in the same subdirectory). 

### Note: This will throw a lot of errors right now. That doesn't matter. Once the for loop completes you can run the next cell.

We will be refactoring the script to eliminate this excessive error reporting.

In [ ]:
html_split = []
article = {}
for file in datafile_list:
    file_name = os.path.splitext(os.path.basename(file))[0]
    sub_dir = results_dir + file_name
    if not os.path.isdir(sub_dir):
        os.makedirs(sub_dir)
    filepath_html = proquest_dir + '/' + file
    with open(filepath_html, 'rt') as infile:
       html_file = infile.read()
    html_split = html_file.split('<div style="margin-bottom:20px;border-bottom:2px solid #ccc;padding-bottom:5px">')
    idx = 0
    for html in html_split:
        padded_id = str(idx).zfill(len(str(len(html_split))))
        article_path = proquest_dir + html
        soup = BeautifulSoup(html, 'html.parser')
        p_tags = soup.find_all('p')
        everything = str(soup)
        body_tags = soup.find_all('body')
        # article title
        try:
            just_second = p_tags[1]
            article_title = just_second.get_text()
            article['title'] = article_title
        except Exception as exc:
            print('! error finding article_title')
            article['title'] = ''
        # doc-id
        try:
            doc_id = str(soup.find('a'))
            doc_id = re.sub('<a name="', '', doc_id)
            doc_id = re.sub('"></a>', '', doc_id)
            article['doc_id'] = doc_id
        except Exception as exc:
            print('! error finding doc-id')
            article['doc_id'] = ''
        # pub
        try:
            pub_title_obj = re.search('<strong>Publication title: </strong>(.+?)</p>', everything)
            pub_title = pub_title_obj.group(1)
            article['pub'] = pub_title
        except Exception as exc:
            print('! error finding pub_title')
            article['pub'] = ''
        # pub-date
        bad_date = '1900-01-01T00:00:00Z'
        try:
            pub_date_obj = re.search('<strong>Publication date: </strong>(.+?)</p>', everything)
            if pub_date_obj:
                pub_date = pub_date_obj.group(1)
            pub_date = re.sub('([a-zA-Z]*)\/([a-zA-Z]*)', r'\1', pub_date)
            date=''
            parse_date = pub_date
            while not date:
                try:
                    date = dateparser.parse(parse_date)
                except Exception as exc:
                    print(' ! error parsing pub_date', exc)
                    parse_date = ' '.join(parse_date.split(' ')[:-1])
                if not parse_date or parse_date.isspace():
                    break
            date_out = date.strftime('%Y-%m-%dT%H:%M:%SZ')
        except Exception as exc:
            print('! error finding pub-date')
            date_out = bad_date
        # volume
        try:
            volume_obj = re.search('<strong>Volume: </strong>(.+?)</p>', everything)
            volume = volume_obj.group(1)
            article['volume'] = volume
        except Exception as exc:
            print('! error finding volume')
            article['volume'] = ''
        # issue
        try:
            issue_obj = re.search('<strong>Issue: </strong>(.+?)</p>', everything)
            issue = issue_obj.group(1)
            article['issue'] = issue
        except Exception as exc:
            print('! error finding issue')
            article['issue'] = ''
        # content
        try:
            just_second_body = body_tags[1]
            article_text = just_second_body.get_text()
            article['content'] = article_text
        except Exception as exc:
            print('! error finding article content')
            article['content'] = ''
        # length
        try:
            length = len(article_text)
            article['length'] = length
        except Exception as exc:
            print('! error finding length')
        # section
        try:
            section_obj = re.search('<strong>Section: </strong>(.+?)</p>', everything)
            section = section_obj.group(1)
            article['section'] = section
        except Exception as exc:
            print('! error finding section')
            article['section'] = ''
        # author
        try:
            just_author = p_tags[2]
            author = just_author.get_text()
            author = re.sub('Author: ', '', author)
            article['author'] = author
        except Exception as exc:
            print('! error finding author')
            article['author'] = ''
        # copyright
        try:
            pub_info_obj = re.search('<strong>Publication info: </strong>(.+?)</p>', everything)
            pub_info = pub_info_obj.group(1)
            article['copyright'] = pub_info
        except Exception as exc:
            print('! error finding copyright')
            article['copyright'] = ''
        # database
        try:
            database_obj = re.search('<strong>Database: </strong>(.+?)</p>', everything)
            database = database_obj.group(1)
            article['database'] = database
        except Exception as exc:
            print('! error finding database')
            article['database'] = 'ProQuest'
        #name
        try:
            pub_title_transform = ''.join(pub_title.split())
            pub_title_transform = re.sub(r'[^\w]','',pub_title_transform)
            name = pub_title_transform + '_' + file_name
        except Exception as exc:
            print('! error adding name')
            article['name'] = ''
        try:
            article['attachment-id'] = ''
            article['name'] = name 
            article['namespace'] = "we1sv2.0"
            article['metapath'] = "Corpus," + name + ",RawData"
            article['pub_date'] = date_out
            article['raw_date'] = pub_date
        except Exception as exc:
            print('! error adding extra keys')
        with open(sub_dir + '/' + file_name + '_' + padded_id + '_.json', 'w') as outfile:
            json.dump(article, outfile, sort_keys=True, indent=2)           
        idx = idx+1  

Delete the first json file in each subdirectory you just created, since it is a list of links.

In [ ]:
for path, subdirs, files in os.walk(results_dir):
    for name in files:
        if '_00_.json' in name:
            first_file = os.path.join(path, name)
            os.remove(first_file)

When WMS and databases are set up: Zip up each subdirectory and delete unzipped directory, send zips of jsons to mongodb. Don't run the below cell for now.

In [ ]:
# for path, subdirs, files in os.walk(results_dir, topdown=True):
    # for subdir in subdirs:
     #  single_subdir = os.path.join(path, subdir)
     #  shutil.make_archive(single_subdir,'zip',single_subdir)
     #  shutil.rmtree(single_subdir)

# then send zips to mongodb

Flatten subdirectory structure to create one `caches/json` directory with all json files in it.

In [ ]:
for path, subdirs, files in os.walk(results_dir):
    for subdir in subdirs:
        single_subdir = os.path.join(path, subdir)
        !cp {single_subdir}/* {results_dir}
        !rm -r {single_subdir}

## NEXT NOTEBOOKS

In [ ]:
from IPython.display import display, HTML

write_project_dir = project_dir.replace('/home/jovyan/', '')
next_link = 'http://harbor.english.ucsb.edu:10000/notebooks/' + write_project_dir

next_link_html = HTML('<h2>Next:</h2><p><ul><li>Go to <a href="' + next_link + '/1_import_data.ipynb" target="_blank"><strong>Notebook 1</a></strong> if you want to further filter, randomly select from, scrub, or de-duplicate your data. If you use notebook 1, do not run the cells with the following titles: BROWSE, LIST, IMPORT. </li><li>If you do not want to filter or de-duplicate your data, go to <a href="' + next_link + '/2_topic_model_data.ipynb" target="_blank"><strong>Notebook 2</a></strong> to model your data.</li></ul></p>')
display(next_link_html)     